In [1]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from Package.HWFourLayers import FourLayerNet
import pickle, math, cv2,os
import matplotlib.pyplot as plt
from PIL import Image

## Function

In [2]:
def init_network():
    with open("Carplate_weight.pkl", 'rb') as f:
        network_weight =pickle.load(f)
    return network_weight

from collections import OrderedDict
from Package.layers import *

def update_weight(self,keys):
    for a in keys:
        self.params[a] = network_weight[a]
    # Layer creation
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2'] = Relu()
    self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])
    self.layers['Relu3'] = Relu()
    self.layers['Affine4'] = Affine(self.params['W4'], self.params['b4'])
    self.lastLayer = SoftmaxWithLoss()

In [3]:
network = FourLayerNet(input_size=1024, hidden_size_1=150, hidden_size_2=100, hidden_size_3=100, output_size=35)
network_weight = init_network()
update_weight(network,network_weight.keys())  # 覆蓋權重

In [4]:
print(network.params['W2'])

[[-0.0225999   0.00421576 -0.00168795 ...  0.11032438 -0.00540068
  -0.0069162 ]
 [-0.00600137  0.01895916 -0.01391903 ...  0.01373453 -0.00971692
  -0.00869058]
 [ 0.11039873  0.02657046  0.06023698 ... -0.0059964   0.05822959
  -0.00231615]
 ...
 [-0.0055406  -0.00175487 -0.01608678 ...  0.00237181  0.01623532
   0.00388047]
 [ 0.00946632  0.00277463  0.0052231  ... -0.01699649  0.00424615
  -0.01455053]
 [ 0.14219182  0.01212878  0.07406871 ... -0.00608848  0.06483303
  -0.00826577]]


## Predict

In [5]:
# 輸入資料
with open ('Carplate_dataset.pkl','rb') as f:
    x=pickle.load(f)
    t=pickle.load(f)
    label=pickle.load(f)

#打亂資料
times = 100
for i in range(times):
    batch_mask = np.random.choice(len(x),len(x))
    x = x[batch_mask]
    t = t[batch_mask]

#資料切割
indices = [int(len(x) * n) for n in [0.7, 0.7 + 0.2]]  # 建立比例值矩陣  訓練:測試:驗證  0.7 0.2 0.1
x_train, x_test, x_val= np.split(x, indices)
t_train, t_test, t_val= np.split(t, indices)

In [6]:
# x ,t = get_data()
x=x_val
t=t_val
accuracy_cnt = 0

for i in range(len(x)):
    y= network.predict(x[i:i+1])
    p= np.argmax(y) # obtain index value of the highest prediction result 
    #print("The "+str(i)+"th image predict as : ", str(label[p]) ," ."+" The actual value should be : "+ str(label[np.argmax(t[i])]))

    if str(label[p]) == str(label[np.argmax(t[i])]):
        accuracy_cnt += 1
            
            
print(" Accuracy: " + str(float(accuracy_cnt)/len(x)))

 Accuracy: 0.45984455958549225
